# Laboratório 07: Avaliação de Sistemas de RI

Antes de responder as perguntas deste laboratório, precisamos importar as extensões que usaremos no decorrer da atividade, bem como criar os índices e implementar os algoritmos binário, TF, BM25 e TF-IDF da última atividade.


*   **Importação das extensões necessárias** 

In [1]:
import csv
import pandas as pd
import numpy as np
import nltk
import re
import collections
import bisect
from nltk.tokenize import RegexpTokenizer
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

*   **Leitura do CSV** 

In [0]:
colecao = pd.read_csv('https://raw.githubusercontent.com/LDVictor/ri_lab_07/master/results.csv')

*   **Criação dos índices** 

In [0]:
tokenizador = RegexpTokenizer(r'([A-Za-zÁáÉéÍíÓóÚúÃãÕõÇçÂâÊê]{3,27})')
stopwords = nltk.corpus.stopwords.words('portuguese') 
indices = {}
n = 0
M = colecao.text.count()

for i in range(len(colecao)):
  texto = colecao.text[i]
  palavras = [palavra for palavra in tokenizador.tokenize(texto.lower())
           if not bool(re.search(r'\d', palavra))
           and palavra not in stopwords and len(palavra) >= 3]   
  for t in palavras:
    if t not in indices.keys():
      indices[t] = []
    indices[t].append(i)
    
for elemento in indices.items():
  d = dict(collections.Counter(elemento[1]))
  indices[elemento[0]] = list(d.items())
  
for palavra in indices:
  k = len(indices[palavra])
  IDF = round(np.log((M+1)/k), 2)
  indices[palavra].append(IDF)  

*   **Implementação dos algoritmos** 

In [0]:
def modeloVetorialBinario(consulta, documento):
  pontos = 0
  tokens_documento = documento.split()
  tokens_consulta = consulta.split()
  
  for token in tokens_consulta:
    pontos += (token in tokens_documento)
 
  return pontos

def modeloVetorialTF(consulta, documento):
  pontos = 0
  tokens_documento = documento.split()
  tokens_consulta = consulta.split()
  
  for token in tokens_consulta:
    pontos += tokens_documento.count(token)
  
  return pontos

def modeloVetorialTFIDF(consulta, documento):
  pontos = 0
  tokens_documento = documento.split()
  tokens_consulta = consulta.split()
  
  for palavra in tokens_consulta:
    cwd = tokens_documento.count(palavra)
    if palavra in indices:
      pontos += cwd * indices[palavra][-1]
  
  return round(pontos, 2)


def modeloVetorialBM25(consulta, documento, k):
  pontos = 0
  tokens_documento = documento.split()
  tokens_consulta = consulta.split()
  
  palavras = [palavra for palavra in tokens_consulta if palavra in tokens_documento]
    
  for palavra in palavras:
    cwd = tokens_documento.count(palavra)
    dfw = 0
    if palavra in indices:
      dfw = len(indices[palavra][:-1])
    pontos += (((k+1) * cwd) / (cwd + k)) * np.log10(((M+1) / dfw)) if dfw != 0 else 0
  
  return round(pontos, 2)

# Questão 1

**Escolha um documento dentre aqueles da base do aluno Bernardi e crie uma consulta que você acha que tem boas chances de recuperar este documento.**

Escolheremos o seguinte documento e consulta:

In [5]:
numero_documento = 52
documento = colecao.loc[numero_documento]
consulta = 'messi barcelona'

coluna = colecao.loc[colecao.url == documento.url]

documento.title

'Comandado por Messi  Barcelona goleia o Lyon e se classifica na Champions'

Agora, criaremos a função para os Top K documentos a partir de cada modelo.


In [0]:
def topKModelos(consulta, k):
  binario = []
  tf = []
  tfidf = []
  bm25 = []
  for i in range(len(colecao)):
    documento = colecao.text[i].lower()
    bisect.insort(binario, (modeloVetorialBinario(consulta, documento), i))
    bisect.insort(tf, (modeloVetorialTF(consulta, documento), i))
    bisect.insort(tfidf, (modeloVetorialTFIDF(consulta, documento), i))
    bisect.insort(bm25, (modeloVetorialBM25(consulta, documento, 20), i))
  
  binario.reverse()
  tf.reverse()
  tfidf.reverse()
  bm25.reverse()
  
  return binario[:k], tf[:k], tfidf[:k], bm25[:k]

In [0]:
top_binario, top_tf, top_tfidf, top_bm25 = topKModelos(consulta, 10)
i_documento = [documento for pontos, documento in top_binario]

Os resultados do modelo serão:

In [8]:
tabela_q1 = pd.DataFrame()

tabela_q1['Binário'] = top_binario
tabela_q1['TF'] = top_tf
tabela_q1['TF-IDF'] = top_tfidf
tabela_q1['BM25'] = top_bm25
tabela_q1.index += 1

tabela_q1

,Binário,TF,TF-IDF,BM25
1,"(2, 52)","(10, 56)","(39.1, 56)","(11.89, 56)"
2,"(1, 174)","(6, 157)","(18.24, 157)","(6.39, 157)"
3,"(1, 157)","(6, 55)","(18.24, 55)","(6.39, 55)"
4,"(1, 138)","(4, 52)","(13.9, 52)","(5.76, 52)"
5,"(1, 136)","(3, 174)","(11.73, 44)","(4.65, 44)"
6,"(1, 129)","(3, 44)","(9.12, 174)","(3.61, 174)"
7,"(1, 124)","(2, 124)","(6.08, 124)","(2.52, 124)"
8,"(1, 67)","(1, 138)","(3.91, 67)","(1.7, 67)"
9,"(1, 64)","(1, 136)","(3.04, 138)","(1.32, 138)"
10,"(1, 56)","(1, 129)","(3.04, 136)","(1.32, 136)"


**Em seguida, avalie os resultados de tal consulta usando a métrica de avaliação Reciprocal Rank.**

Primeiro, vamos definir a função Reciprocal Rank, que deve retornar sua métrica de avaliação.

In [0]:
def reciprocalRank(tuplas, id_documento):
  n = 1.0;
  
  for r, documento in tuplas:
    if documento == id_documento:
      return [round(1 / n, 2)]
    else:
      n += 1

Agora, iremos verificar os resultados do Reciprocal Rank.

In [10]:
tabela_reciprocal_rank = pd.DataFrame()
tabela_reciprocal_rank['Binário'] = reciprocalRank(tabela_q1['Binário'], numero_documento)
tabela_reciprocal_rank['TF'] = reciprocalRank(tabela_q1['TF'], numero_documento)
tabela_reciprocal_rank['TF-IDF'] = reciprocalRank(tabela_q1['TF-IDF'], numero_documento)
tabela_reciprocal_rank['BM25'] = reciprocalRank(tabela_q1['BM25'], numero_documento)
tabela_reciprocal_rank.index += 1

tabela_reciprocal_rank

,Binário,TF,TF-IDF,BM25
1,1.0,0.25,0.25,0.25


A métrica Reciprocal Rank varia no intervalo entre 0 e 1. Os valores que calculamos indicam que o modelo binário assumiu valor máximo na busca pelos documentos específicos que foram fornecidos no enunciado da atividade.

# Questão 2

**A partir do gabarito fornecido em OBS1, calcule o MAP para cada algoritmo abaixo e aponte qual obteve o melhor resultado. Para os cálculos do MAP, considere que um documento é relevante para uma dada consulta se este documento estiver entre os documentos do gabarito para essa consulta, senão ele deve ser considerado irrelevante.**

Inicialmente, iremos importar o gabarito fornecido no enunciado da atividade.

In [0]:
colecao_obs1 = pd.read_json('https://raw.githubusercontent.com/LDVictor/ri_lab_07/master/results_final.json')
feedback = {colecao_obs1['query'][i]:colecao_obs1['docs'][i] for i in range(10)}

Após isso, definiremos as funções auxiliares que serão usadas na função MAP.

In [0]:
def indicesDocumento(modelo):
  return [documento for pontos, documento in modelo]

def intersecao(a, b):
  return [elemento for elemento in a if elemento in b]

def calculaAP(consulta):
  documentos_relevantes = []

  for info_documento in feedback[consulta]:
    coluna = colecao.loc[colecao.url == info_documento['URL']]
    documentos_relevantes.append(coluna.index[0])
  
  binario, tf, tfidf, bm25 = topKModelos(consulta, 5)
  
  binario = indicesDocumento(binario)
  tf = indicesDocumento(tf)
  tfidf = indicesDocumento(tfidf)
  bm25 = indicesDocumento(bm25)
  
  ap_binario = len(intersecao(binario, documentos_relevantes)) / len(binario)
  ap_tf = len(intersecao(tf, documentos_relevantes)) / len(tf)
  ap_tfidf = len(intersecao(tfidf, documentos_relevantes)) / len(tfidf)
  ap_bm25 = len(intersecao(bm25, documentos_relevantes)) / len(bm25)
  
  return ap_binario, ap_tf, ap_tfidf, ap_bm25

Agora, podemos definir a função para calcular o MAP.

In [0]:
def calculaMAP(consultas):
  soma_binario = 0
  soma_tf = 0
  soma_tfidf = 0
  soma_bm25 = 0
  
  for consulta in consultas:
    ap_binario, ap_tf, ap_tfidf, ap_bm25 = calculaAP(consulta)
    soma_binario += ap_binario
    soma_tf += ap_tf
    soma_tfidf += ap_tfidf
    soma_bm25 += ap_bm25
  
  map_binario = round(soma_binario / len(consultas), 2)
  map_tf = round(soma_tf / len(consultas), 2)
  map_tfidf = round(soma_tfidf / len(consultas), 2)
  map_bm25 = round(soma_bm25 / len(consultas), 2)
  
  return map_binario, map_tf, map_tfidf, map_bm25

Finalmente, podemos calcular o MAP para cada um dos quatro algoritmos a partir da nova coleção.

In [0]:
map_binario, map_tf, map_tfidf, map_bm25 = calculaMAP(feedback.keys())



*   Rep. Binária

In [15]:
map_binario

0.1



*   TF

In [16]:
map_tf

0.02


*   TF-IDF

In [17]:
map_tfidf

0.18


*   BM25

In [18]:
map_bm25

0.18

O valor de medida do MAP sempre deve estar no invervalo de 0 a 1. Os valores que calculamos indicam que todos os modelos possuem uma baixa precisão na busca pelos documentos específicos que foram fornecidos no enunciado da atividade.

# Questão 3

**Repita Q2 usando a avaliação multi-nível DCG. Utilize o campo "level" do gabarito para o cálculo do DCG e do idealDCG. Use uma janela de 5 documentos.**

De início, vamos implementar a função de avaliação DCG e DCG ideal.

In [0]:
def calculaDCG(modelo, niveis):
  dcg = 0.0
  for i in range(1,len(modelo)+1):
    doc = modelo[i-1]
    nivel = getNivel(doc, niveis)
    dcg += (2^nivel) / np.log2(i + 1.0)
    
  return dcg

def modelosDCG(consulta):
  documentos_relevantes = {}

  for info_documento in feedback[consulta]:
    coluna = colecao.loc[colecao.url == info_documento['URL']]
    documentos_relevantes[coluna.index[0]] = info_documento['level']
    
  binario, tf, tfidf, bm25 = topKModelos(consulta, 5)
  binario, tf, tfidf, bm25 = todosOsDocumentos(binario, tf, tfidf, bm25)
  
  dcg_binario = round(calculaDCG(binario, setNivel(binario, documentos_relevantes)),2)
  dcg_tf = round(calculaDCG(tf, setNivel(tf, documentos_relevantes)),2)
  dcg_tfidf = round(calculaDCG(tfidf, setNivel(tfidf, documentos_relevantes)),2)
  dcg_bm25 = round(calculaDCG(bm25, setNivel(bm25, documentos_relevantes)),2)
  
  return dcg_binario, dcg_tf, dcg_tfidf, dcg_bm25


def modelosDCGIdeal(consulta):
  documentos_relevantes = {}

  for info_documento in feedback[consulta]:
    coluna = colecao.loc[colecao.url == info_documento['URL']]
    documentos_relevantes[coluna.index[0]] = info_documento['level']
    
  binario, tf, tfidf, bm25 = topKModelos(consulta, 5)
  binario = indicesDocumento(binario)
  tf = indicesDocumento(tf)
  tfidf = indicesDocumento(tfidf)
  bm25 = indicesDocumento(bm25)
  
  niveis_binario, niveis_tf, niveis_tfidf, niveis_bm25 = todosOsNiveis(binario, tf, tfidf, bm25, documentos_relevantes)
  
  binario, tf, tfidf, bm25 = extraiDocumentos(niveis_binario, niveis_tf, niveis_tfidf, niveis_bm25)
  
  idcg_binario = round(calculaDCG(binario, niveis_binario), 2)
  idcg_tf = round(calculaDCG(tf, niveis_tf), 2)
  idcg_tfidf = round(calculaDCG(tfidf, niveis_tfidf), 2)
  idcg_bm25 = round(calculaDCG(bm25, niveis_bm25), 2)
  
  return idcg_binario, idcg_tf, idcg_tfidf, idcg_bm25

Além disso, definiremos também as seguintes funções auxiliares:

In [0]:
def getNivel(d, l):
  for level,doc in l:
    if doc == d:
      return level
    
def setNivel(m, d):
  modelo = [(0, documento) for documento in m if documento not in d]
  dic = [(v, k) for k, v in d.items()]
  res = modelo + dic
  res.sort(reverse=True)
  return res

def todosOsDocumentos(binario, tf, tfidf, bm25):
  return indicesDocumento(binario), indicesDocumento(tf), indicesDocumento(tfidf), indicesDocumento(bm25)
    
def todosOsNiveis(binario, tf, tfidf, bm25, rd):
  return setNivel(binario, rd), setNivel(tf, rd), setNivel(tfidf, rd), setNivel(bm25, rd)

def extraiDocumentos(binario, tf, tfidf, bm25):
  return [documento for nivel, documento in binario], [documento for nivel, documento in tf], [documento for nivel, documento in tfidf], [documento for nivel, documento in bm25]

Agora, vamos definir os valores do DCG a partir dos modelos.

In [0]:
resultados_consultas = {}
for consulta in feedback.keys():
  dcg_binario, dcg_tf, dcg_tfidf, dcg_bm25 = modelosDCG(consulta)
  idcg_binario, idcg_tf, idcg_tfidf, idcg_bm25 = modelosDCGIdeal(consulta)
  
  binario = (dcg_binario, idcg_binario)
  tf = (dcg_tf, idcg_tf)
  tfidf = (dcg_tfidf, idcg_tfidf)
  bm25 = (dcg_bm25, idcg_bm25)
  
  resultados = [binario, tf, tfidf, bm25]
  
  resultados_consultas[consulta] = resultados

In [22]:
tabela_q3 = pd.DataFrame()

tabela_q3['Consulta'] = feedback.keys()
tabela_q3['Binário'] = [resultados_consultas[consulta][0] for consulta in feedback.keys()]
tabela_q3['TF'] = [resultados_consultas[consulta][1] for consulta in feedback.keys()]
tabela_q3['TF-IDF'] = [resultados_consultas[consulta][2] for consulta in feedback.keys()]
tabela_q3['BM25'] = [resultados_consultas[consulta][3] for consulta in feedback.keys()]
tabela_q3.index += 1

tabela_q3

,Consulta,Binário,TF,TF-IDF,BM25
1,território palestino,"(5.9, 15.65)","(5.9, 15.65)","(5.51, 14.98)","(5.51, 14.98)"
2,recessão mundial,"(11.58, 14.98)","(9.77, 14.98)","(9.77, 14.98)","(9.77, 14.98)"
3,ditadura militar,"(5.9, 17.17)","(5.9, 17.17)","(5.9, 17.17)","(5.9, 17.17)"
4,muro das lamentações,"(18.08, 19.29)","(5.9, 21.3)","(19.29, 19.29)","(19.29, 19.29)"
5,brasil e argentina,"(8.9, 17.5)","(5.9, 18.17)","(7.79, 17.5)","(7.79, 17.5)"
6,golpe militar,"(5.9, 20.67)","(5.9, 20.67)","(8.4, 20.04)","(8.4, 20.04)"
7,governo bolsonaro,"(5.9, 16.54)","(5.9, 16.54)","(5.9, 16.54)","(5.9, 16.54)"
8,ministro da economia,"(5.9, 17.17)","(5.9, 17.17)","(5.9, 17.17)","(5.9, 17.17)"
9,prisão de Temer,"(5.9, 13.43)","(5.9, 13.43)","(10.29, 12.05)","(10.29, 12.05)"
10,Congresso Nacional,"(5.9, 9.65)","(5.9, 9.65)","(5.9, 9.65)","(5.9, 9.65)"



Na tabela, temos a disposição dos resultados encontrados para o DCG comum e DCG ideal, nas tuplas para os modelos que usamos (Binário, TF, TF-IDF e BM25).